# 🧹 Limpieza de Datos - ZonaProp Scraper

Este notebook limpia y procesa los datos scraped de ZonaProp paso a paso.

**Objetivo**: Transformar los datos raw en un dataset limpio y estructurado para análisis.

## 📚 1. Importar Librerías

In [6]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías importadas correctamente")

✅ Librerías importadas correctamente


## 📁 2. Cargar Datos Raw

In [7]:
# Cargar datos raw
df_raw = pd.read_csv("../data/zonaprop_raw.csv")

print(f"📊 Dataset cargado: {len(df_raw)} propiedades")
print(f"📋 Columnas: {list(df_raw.columns)}")
print(f"💾 Tamaño: {df_raw.shape}")

# Vista previa
df_raw.head(3)

📊 Dataset cargado: 30 propiedades
📋 Columnas: ['titulo', 'precio', 'descripcion', 'ubicacion']
💾 Tamaño: (30, 4)


,titulo,precio,descripcion,ubicacion
0,102 m² tot.\n4 amb.\n2 dorm.\n2 baños,$ 650.000,"Inmobiliaria liprandi, te ofrece: Excelente de...","San Salvador, Córdoba"
1,98 m² tot.\n6 amb.\n2 baños\n1 coch.,$ 1.100.000,Grupo Seis Negocios Inmobiliarios alquila depa...,"Villa Cabrera, Córdoba"
2,50 m² tot.\n2 amb.\n1 dorm.\n1 baño,$ 600.000,Carlota Montrasi Servicios Inmobiliarios te of...,"Villa Carlos Paz, Córdoba"


## 🔍 3. Exploración Inicial

In [8]:
# Información general
print("📋 INFO GENERAL:")
print(f"Filas: {len(df_raw)}")
print(f"Columnas: {len(df_raw.columns)}")
print(f"Valores nulos: {df_raw.isnull().sum().sum()}")

print("\n🔎 VALORES NULOS POR COLUMNA:")
print(df_raw.isnull().sum())

print("\n📏 TIPOS DE DATOS:")
print(df_raw.dtypes)

print("\n🏠 EJEMPLO DE TÍTULO CON FORMATO:")
print("Título original:")
print(repr(df_raw['titulo'].iloc[0]))  # Ver formato exacto con \n
print("\nTítulo formateado:")
print(df_raw['titulo'].iloc[0])  # Ver cómo se ve normalmente

📋 INFO GENERAL:
Filas: 30
Columnas: 4
Valores nulos: 0

🔎 VALORES NULOS POR COLUMNA:
titulo         0
precio         0
descripcion    0
ubicacion      0
dtype: int64

📏 TIPOS DE DATOS:
titulo         object
precio         object
descripcion    object
ubicacion      object
dtype: object

🏠 EJEMPLO DE TÍTULO CON FORMATO:
Título original:
'102 m² tot.\n4 amb.\n2 dorm.\n2 baños'

Título formateado:
102 m² tot.
4 amb.
2 dorm.
2 baños


## 💰 4. Limpieza de Precios

In [9]:
# Crear copia para trabajar
df = df_raw.copy()

def limpiar_precio(precio_str):
    """
    Convierte string de precio a número
    Ej: '$ 650.000' -> 650000
    """
    if pd.isna(precio_str) or precio_str == "Sin precio":
        return None
    
    # Remover $ y espacios, convertir puntos a nada
    precio_limpio = re.sub(r'[^\d]', '', str(precio_str))
    
    try:
        return int(precio_limpio)
    except:
        return None

# Aplicar limpieza
df['precio_numerico'] = df['precio'].apply(limpiar_precio)

print("🧹 ANTES vs DESPUÉS - PRECIOS:")
comparacion_precios = df[['precio', 'precio_numerico']].head(5)
print(comparacion_precios)

print(f"\n📊 ESTADÍSTICAS DE PRECIOS:")
print(df['precio_numerico'].describe())

🧹 ANTES vs DESPUÉS - PRECIOS:
        precio  precio_numerico
0    $ 650.000           650000
1  $ 1.100.000          1100000
2    $ 600.000           600000
3    $ 500.000           500000
4    $ 550.000           550000

📊 ESTADÍSTICAS DE PRECIOS:
count    3.000000e+01
mean     6.628333e+05
std      2.461976e+05
min      3.200000e+05
25%      5.000000e+05
50%      6.000000e+05
75%      7.725000e+05
max      1.300000e+06
Name: precio_numerico, dtype: float64


## 🏠 5. Extracción de Características del Título

In [10]:
def extraer_caracteristicas(titulo):
    """
    Extrae metros cuadrados, ambientes, dormitorios, baños y cocheras del título
    Formato esperado: '102 m² tot.\n4 amb.\n2 dorm.\n2 baños'
    """
    if pd.isna(titulo):
        return None, None, None, None, None
    
    titulo_str = str(titulo)
    
    # Metros cuadrados (ej: "102 m² tot.")
    metros_match = re.search(r'(\d+)\s*m²', titulo_str)
    metros = int(metros_match.group(1)) if metros_match else None
    
    # Ambientes (ej: "4 amb.")
    amb_match = re.search(r'(\d+)\s*amb', titulo_str)
    ambientes = int(amb_match.group(1)) if amb_match else None
    
    # Dormitorios (ej: "2 dorm.")
    dorm_match = re.search(r'(\d+)\s*dorm', titulo_str)
    dormitorios = int(dorm_match.group(1)) if dorm_match else None
    
    # Baños (ej: "2 baños")
    bano_match = re.search(r'(\d+)\s*baño', titulo_str)
    banos = int(bano_match.group(1)) if bano_match else None
    
    # Cocheras (ej: "1 coch.")
    coch_match = re.search(r'(\d+)\s*coch', titulo_str)
    cocheras = int(coch_match.group(1)) if coch_match else 0
    
    return metros, ambientes, dormitorios, banos, cocheras

# Aplicar extracción
print("🔍 Extrayendo características del título...")
caracteristicas = df['titulo'].apply(extraer_caracteristicas)

# Crear columnas separadas
df['metros_cuadrados'] = [x[0] for x in caracteristicas]
df['ambientes'] = [x[1] for x in caracteristicas]
df['dormitorios'] = [x[2] for x in caracteristicas]
df['banos'] = [x[3] for x in caracteristicas]
df['cocheras'] = [x[4] for x in caracteristicas]

print("✅ Características extraídas correctamente!")

# Mostrar ejemplos de extracción
print("\n🔍 EJEMPLOS DE EXTRACCIÓN:")
ejemplo = df[['titulo', 'metros_cuadrados', 'ambientes', 'dormitorios', 'banos', 'cocheras']].head(3)
for idx, row in ejemplo.iterrows():
    print(f"\n{idx+1}. Título original:")
    # Mostrar título con saltos de línea reemplazados por | para mayor claridad
    titulo_visual = row['titulo'].replace('\n', ' | ')
    print(f"   {titulo_visual}")
    print(f"   📐 {row['metros_cuadrados']}m² | 🏠 {row['ambientes']} amb | 🛏️ {row['dormitorios']} dorm | 🚿 {row['banos']} baños | 🚗 {row['cocheras']} coch")

# Resumen de completitud
print("\n📊 COMPLETITUD DE EXTRACCIÓN:")
caracteristicas_cols = ['metros_cuadrados', 'ambientes', 'dormitorios', 'banos', 'cocheras']
for col in caracteristicas_cols:
    completitud = df[col].notna().sum()
    porcentaje = completitud / len(df) * 100
    print(f"{col}: {completitud}/{len(df)} ({porcentaje:.1f}%)")

🔍 Extrayendo características del título...
✅ Características extraídas correctamente!

🔍 EJEMPLOS DE EXTRACCIÓN:

1. Título original:
   102 m² tot. | 4 amb. | 2 dorm. | 2 baños
   📐 102m² | 🏠 4.0 amb | 🛏️ 2.0 dorm | 🚿 2 baños | 🚗 0 coch

2. Título original:
   98 m² tot. | 6 amb. | 2 baños | 1 coch.
   📐 98m² | 🏠 6.0 amb | 🛏️ nan dorm | 🚿 2 baños | 🚗 1 coch

3. Título original:
   50 m² tot. | 2 amb. | 1 dorm. | 1 baño
   📐 50m² | 🏠 2.0 amb | 🛏️ 1.0 dorm | 🚿 1 baños | 🚗 0 coch

📊 COMPLETITUD DE EXTRACCIÓN:
metros_cuadrados: 30/30 (100.0%)
ambientes: 29/30 (96.7%)
dormitorios: 29/30 (96.7%)
banos: 30/30 (100.0%)
cocheras: 30/30 (100.0%)


## 📍 6. Procesamiento de Ubicaciones

In [11]:
# Separar barrio y ciudad de la ubicación
df['barrio'] = df['ubicacion'].str.split(',').str[0].str.strip()
df['ciudad'] = df['ubicacion'].str.split(',').str[1].str.strip().fillna('Córdoba')

print("📍 UBICACIONES PROCESADAS:")
print(f"Barrios únicos: {df['barrio'].nunique()}")
print(f"Ciudades únicas: {df['ciudad'].nunique()}")

print("\n🏘️ TOP 10 BARRIOS MÁS FRECUENTES:")
print(df['barrio'].value_counts().head(10))

print("\n🏙️ DISTRIBUCIÓN POR CIUDAD:")
print(df['ciudad'].value_counts())

📍 UBICACIONES PROCESADAS:
Barrios únicos: 14
Ciudades únicas: 3

🏘️ TOP 10 BARRIOS MÁS FRECUENTES:
barrio
Nueva Córdoba       9
General Paz         5
Alto Alberdi        2
Córdoba             2
Manantiales         2
Centro              2
Villa Carlos Paz    1
Villa Cabrera       1
San Salvador        1
Don Bosco           1
Name: count, dtype: int64

🏙️ DISTRIBUCIÓN POR CIUDAD:
ciudad
Córdoba            28
Valle Escondido     1
Villa Allende       1
Name: count, dtype: int64


## 💡 7. Variables Derivadas

In [12]:
# Crear precio por metro cuadrado
df['precio_por_m2'] = df['precio_numerico'] / df['metros_cuadrados']

# Categorizar precios
def categorizar_precio(precio):
    if pd.isna(precio):
        return "Sin datos"
    elif precio < 500000:
        return "Económico"
    elif precio < 800000:
        return "Medio"
    elif precio < 1200000:
        return "Alto"
    else:
        return "Premium"

df['categoria_precio'] = df['precio_numerico'].apply(categorizar_precio)

# Categorizar tamaños
def categorizar_tamano(metros):
    if pd.isna(metros):
        return "Sin datos"
    elif metros < 50:
        return "Pequeño"
    elif metros < 80:
        return "Mediano"
    elif metros < 120:
        return "Grande"
    else:
        return "Muy grande"

df['categoria_tamano'] = df['metros_cuadrados'].apply(categorizar_tamano)

print("💡 VARIABLES DERIVADAS CREADAS:")
print(f"✅ precio_por_m2: Promedio ${df['precio_por_m2'].mean():.0f}/m²")
print(f"✅ categoria_precio: {df['categoria_precio'].value_counts().to_dict()}")
print(f"✅ categoria_tamano: {df['categoria_tamano'].value_counts().to_dict()}")

💡 VARIABLES DERIVADAS CREADAS:
✅ precio_por_m2: Promedio $10156/m²
✅ categoria_precio: {'Medio': 17, 'Alto': 6, 'Económico': 6, 'Premium': 1}
✅ categoria_tamano: {'Pequeño': 11, 'Grande': 8, 'Mediano': 8, 'Muy grande': 3}


## 💾 8. Guardar Dataset Limpio

In [13]:
# Seleccionar columnas importantes para el dataset final
columnas_finales = [
    'titulo', 'precio_numerico', 'precio_por_m2', 'categoria_precio',
    'metros_cuadrados', 'categoria_tamano', 'ambientes', 'dormitorios', 
    'banos', 'cocheras', 'barrio', 'ciudad', 'descripcion'
]

# Filtrar datos válidos (con precio y metros cuadrados)
df_final = df[columnas_finales].dropna(subset=['precio_numerico', 'metros_cuadrados'])

# Guardar dataset limpio
output_path = "../data/zonaprop_clean.csv"
df_final.to_csv(output_path, index=False, encoding='utf-8')

print(f"✅ Dataset limpio guardado en: {output_path}")
print(f"📊 Propiedades guardadas: {len(df_final)}")
print(f"📋 Columnas guardadas: {len(df_final.columns)}")
print("\n🎉 ¡LIMPIEZA DE DATOS COMPLETADA!")

# Mostrar resumen estadístico
print("\n📊 RESUMEN ESTADÍSTICO:")
estadisticas = df_final[['precio_numerico', 'precio_por_m2', 'metros_cuadrados', 'ambientes', 'dormitorios']].describe()
print(estadisticas)

print("\n🏷️ DISTRIBUCIÓN POR CATEGORÍA:")
print("Precios:")
print(df_final['categoria_precio'].value_counts())
print("\nTamaños:")
print(df_final['categoria_tamano'].value_counts())

# Vista previa del dataset final
print("\n👀 VISTA PREVIA DEL DATASET LIMPIO:")
preview_cols = ['precio_numerico', 'metros_cuadrados', 'ambientes', 'dormitorios', 'banos', 'barrio']
print(df_final[preview_cols].head())

✅ Dataset limpio guardado en: ../data/zonaprop_clean.csv
📊 Propiedades guardadas: 30
📋 Columnas guardadas: 13

🎉 ¡LIMPIEZA DE DATOS COMPLETADA!

📊 RESUMEN ESTADÍSTICO:
       precio_numerico  precio_por_m2  metros_cuadrados  ambientes  \
count     3.000000e+01      30.000000         30.000000  29.000000   
mean      6.628333e+05   10156.406747         71.066667   2.758621   
std       2.461976e+05    3572.893031         34.826598   1.353703   
min       3.200000e+05    4473.684211         40.000000   2.000000   
25%       5.000000e+05    7506.105834         45.000000   2.000000   
50%       6.000000e+05   10185.185185         54.500000   2.000000   
75%       7.725000e+05   11916.666667         90.000000   3.000000   
max       1.300000e+06   22222.222222        190.000000   8.000000   

       dormitorios  
count    29.000000  
mean      1.448276  
std       0.506120  
min       1.000000  
25%       1.000000  
50%       1.000000  
75%       2.000000  
max       2.000000  

🏷️ DISTRIBU